In [1]:
from benchmarks.qrom import QROM_Schedule
import numpy as np

from swiper.simulator import DecodingSimulator
import swiper.plot as plotter

In [2]:
qrom15 = QROM_Schedule([np.arange(15)], 15, 15)

In [3]:
d=7
decoding_time = 2*d
speculation_time = 0
speculation_accuracy = 1

simulator = DecodingSimulator(d, lambda _: decoding_time, speculation_time, speculation_accuracy, speculation_mode='integrated')

In [4]:
success, device_data, window_data, decoding_data = simulator.run(
    schedule=qrom15.schedule,
    scheduling_method='sliding',
    max_parallel_processes=None,
    progress_bar=True,
    save_animation_frames=False
)

Surface code rounds: 1745it [00:01, 1193.11it/s]


In [ ]:
plotter.plot_device_schedule_trace(device_data, hide_z_ticks=True)